In [1]:
import pandas as pd
import numpy as np
import os
import pygeohash as pgh
from math import *
pjoin = os.path.join

In [210]:
#data load
datapath = '/Users/yefeichen/Database/location_recommender_system/'
cfile = ['dnb_pa.csv','dnb_sf.csv','dnb_sj.csv','dnb_Los_Angeles.csv','dnb_New_York.csv']
lfile = 'location_scorecard_190912.csv'
clfile = ['PA.csv','SF.csv','SJ.csv','LA.csv','NY.csv']
cityname = ['Palo Alto','San Francisco','San Jose','Los Angeles', 'New York']
ssfile = ['ww_PA_similarity_20191113.csv','ww_SF_similarity_20191113.csv','ww_SJ_similarity_20191113.csv',
          'ww_LA_similarity_20191113.csv','ww_NY_similarity_20191113.csv']
# lfile_app = ['PA_comp_loc_score.csv','SF_comp_loc_score.csv','SJ_comp_loc_score.csv']
# testfile = 'train_val_test_location_company_all.csv'
wework_location_only = True


for ind_city in range(5):
    comp_feat = pd.read_csv(pjoin(datapath,cfile[ind_city]))
    comp_loc = pd.read_csv(pjoin(datapath,clfile[ind_city]))
    loc_feat = pd.read_csv(pjoin(datapath,lfile))

    #if we only focus on buildings with companies inside
    loc_feat = loc_feat.merge(comp_loc[['atlas_location_uuid']].groupby('atlas_location_uuid').first().reset_index(), on='atlas_location_uuid',how='inner',suffixes=['','_right'])

    print('Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>')
    print('Filtering')
    #Global filter: master degree!
    global_ft = global_filter(loc_feat=loc_feat)
    sub_loc_feat = global_ft.city_filter(city_name=cityname[ind_city]).end()


    if wework_location_only:
        sub_loc_feat_ww = sub_loc_feat[sub_loc_feat['is_wework']==True]
        sub_comp_loc = pd.merge(comp_loc,sub_loc_feat_ww[['atlas_location_uuid']],on='atlas_location_uuid',how='inner',suffixes=['','_right'])
    else:
        sub_comp_loc = pd.merge(comp_loc, sub_loc_feat[['atlas_location_uuid']], on='atlas_location_uuid',
                                how='inner', suffixes=['', '_right'])

    print('Reasoning')
    #Reason 1:
    print('Customized Reason')
    matching_col = 'major_industry_category'
    # matching_col = 'primary_sic_2_digit_v2'
    recall_com1 = sub_rec_similar_company(comp_feat=comp_feat, comp_loc=sub_comp_loc,
                                          matching_col=matching_col,reason_col_name='reason')
    sub_pairs = recall_com1.get_candidate_location_for_company(query_comp_feat=comp_feat,reason='like')
    sub_pairs['reason'] = sub_pairs.apply(lambda x: 'suitable for '+ x[matching_col],axis=1)
    print(sub_pairs.shape)

    #Reason2:
    print('General Reason')
    recall_com2 = sub_rec_condition(sub_loc_feat)
    sub_loc_recall_com2 = recall_com2.exfiltering('num_fitness_gyms', percentile=0.5, reason='Enough GYM')
    sub_loc_recall_com3 = recall_com2.exfiltering('num_drinking_places', percentile=0.5,
                                                  reason='Entertainment Available')
    sub_loc_recall_com4 = recall_com2.exfiltering('num_eating_places', percentile=0.5, reason='Easy for lunch')
    print('recall_location_size: %d, %d, %d'%(len(sub_loc_recall_com2), len(sub_loc_recall_com3), len(sub_loc_recall_com4.shape)))

    print('Merging general reasons')
    sub_loc_recall = pd.concat([sub_loc_recall_com2, sub_loc_recall_com3, sub_loc_recall_com4], axis=0)
    sub_loc_recall = merge_rec_reason_rowise(sub_loc_recall, group_cols=['atlas_location_uuid'],
                                             merge_col='reason')
    if wework_location_only:
        sub_loc_recall = sub_loc_recall.merge(sub_loc_feat_ww[['atlas_location_uuid']],on='atlas_location_uuid',how='inner',suffixes=['','_right'])
    print('sub_loc_recall sized %d' % len(sub_loc_recall))
    sub_loc_recall.head()
    
    #### attach these with scores
    topk=300
    sspd = pd.read_csv(pjoin(datapath,ssfile[ind_city]),index_col=0)

    sample_sspd = sspd.groupby('atlas_location_uuid').apply(lambda x: x.nlargest(topk,['similarity'])).reset_index(drop=True)

    print(len(sample_sspd))

    sample_sspd = pd.merge(sample_sspd,sub_loc_recall[['atlas_location_uuid','reason']],on='atlas_location_uuid',how='left',suffixes=['','_right']).reset_index(drop=True)

    sample_sspd = pd.merge(sample_sspd,sub_pairs,on=['atlas_location_uuid','duns_number'],how='left',suffixes=['','_right'])

    sample_sspd = sample_sspd[sample_sspd['reason'].notnull() | sample_sspd['reason_right'].notnull()]

    sample_sspd[['reason','reason_right']] = sample_sspd[['reason','reason_right']].fillna('')

    sample_sspd = merge_rec_reason_colwise(sample_sspd,cols=['reason','reason_right'],dst_col='reason')


    sample_sspd = reason_json_format(sample_sspd,col_name='reason')

    sample_sspd['duns_number'] = sample_sspd['duns_number'].astype(int)
    sample_sspd = sample_sspd.rename(columns={
        "reason":"note","duns_number":"company_id"
    })
    sample_sspd['building_id'] = sample_sspd['atlas_location_uuid'].apply(lambda x: hash(x))
    sample_sspd['algorithm']= 'model_deep_and_wide'

    col_list = ['company_id','building_id','similarity','note','atlas_location_uuid','algorithm']
    sample_sspd = sample_sspd[col_list]

    print(len(sample_sspd))

    sample_sspd.to_csv('sub_'+ssfile[ind_city])
    # sspd[sspd['reason_right'].isnull()]


Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>
Filtering
Reasoning
Customized Reason
(6988, 4)
General Reason
recall_location_size: 184, 253, 2
Merging general reasons
sub_loc_recall sized 1
300
300


/Users/yefeichen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: FutureWarning: `item` has been deprecated and will be removed in a future version


Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>
Filtering
Reasoning
Customized Reason
(988917, 4)
General Reason
recall_location_size: 721, 731, 2
Merging general reasons
sub_loc_recall sized 15
5400
5335
Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>
Filtering
Reasoning
Customized Reason
(91497, 4)
General Reason
recall_location_size: 314, 343, 2
Merging general reasons
sub_loc_recall sized 3
900
900
Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>
Filtering
Reasoning
Customized Reason
(1268040, 4)
General Reason
recall_location_size: 580, 580, 2
Merging general reasons
sub_loc_recall sized 9
3900
3801
Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>
Filtering
Reasoning
Customized Reason
(10428396, 4)
General Reason
recall_location_size: 1137, 1146, 2
Merging general reasons
sub_loc_recall sized 51
23100
21864


In [209]:
def generate_loc_type(comp_feat, comp_loc, matching_col):
    # matching_col = 'major_industry_category'
    comp_type = comp_feat[['duns_number', matching_col]]
    comp_type_location = pd.merge(comp_type, comp_loc[['duns_number', 'atlas_location_uuid']], on='duns_number',
                                  how='inner')

    loc_type = comp_type_location.groupby(['atlas_location_uuid', matching_col]).first().reset_index()[
        ['atlas_location_uuid', matching_col]]
    return loc_type


class sub_rec_similar_company(object):
    def __init__(self, comp_feat, comp_loc, matching_col,reason_col_name='reason'):
        """
        comp_feat: original company information
        comp_loc: company-location affinities of a certain city
        matching_col = 'major_industry_category' big category
                    or 'primary_sic_2_digit' more detailed category
        """
        self.comp_feat = comp_feat
        self.comp_loc = comp_loc
        self.matching_col = matching_col
        self.reason_col_name = reason_col_name
        self.loc_type = generate_loc_type(comp_feat, comp_loc, matching_col)

    def get_candidate_location_for_company(self, query_comp_feat,reason='similar company inside'):
        sub_pairs = pd.merge(query_comp_feat[['duns_number', self.matching_col]], self.loc_type, on=self.matching_col,
                             how='left', suffixes=['', '_right'])
        sub_pairs = sub_pairs[sub_pairs['atlas_location_uuid'].notnull()]#sometimes a company may have no location to recommend
        sub_pairs[self.reason_col_name] = reason
        return sub_pairs


class global_filter(object):
    def __init__(self, loc_feat):
        self.loc_feat = loc_feat

    def filtering(self, key_column, percentile=0.2, mode='gt'):
        val = self.loc_feat[[key_column]].quantile(q=percentile).item()
        if mode == 'gt':
            sub_loc = self.loc_feat[self.loc_feat[key_column] >= val]
        else:
            sub_loc = self.loc_feat[self.loc_feat[key_column] <= val]

        self.loc_feat = sub_loc.reset_index(drop=True)
        return self

    def city_filter(self, city_name, key_column='city'):
        self.loc_feat = self.loc_feat[self.loc_feat[key_column] == city_name].reset_index(drop=True)
        return self

    def exfiltering(self, loc_feat, key_column, percentile=0.2, mode='gt'):
        val = loc_feat[[key_column]].quantile(q=percentile).item()
        if mode == 'gt':
            sub_loc = self.loc_feat[self.loc_feat[key_column] >= val]
        else:
            sub_loc = self.loc_feat[self.loc_feat[key_column] <= val]

        return sub_loc.reset_index(drop=True)

    def end(self):
        return self.loc_feat


class sub_rec_condition(object):
    def __init__(self, loc_feat):
        """
        comp_loc: company-location affinities of a certain city
        cond_col = column of location used for filtering
        """
        self.loc_feat = loc_feat
        self.cond_col = []
        self.reason = []

    def filtering(self, cond_col, percentile=0.5, reason='many things'):
        self.cond_col.append(cond_col)
        val = self.loc_feat[[cond_col]].quantile(q=percentile).item()
        if max(val, 10):
            self.loc_feat = self.loc_feat[self.loc_feat[cond_col] >= val].reset_index(drop=True)
            self.reason.append(reason)
        return self

    def exfiltering(self, cond_col, percentile=0.6, reason='many things'):
        self.cond_col.append(cond_col)
        val = self.loc_feat[[cond_col]].quantile(q=percentile).item()
        if max(val, 10):
            sub_loc = self.loc_feat[self.loc_feat[cond_col] >= val].reset_index(drop=True)
        sub_loc['reason'] = reason
        return sub_loc[['atlas_location_uuid', 'reason']]

    def end(self):
        return self.loc_feat

#======================================================================================================================
def ab(df):
    return ','.join(df.values)


def merge_rec_reason_rowise(sub_pairs, group_cols: list, merge_col: str):
    return sub_pairs.groupby(group_cols)[merge_col].apply(ab).reset_index()


def merge_rec_reason_colwise(sub_pairs, cols=['reason1', 'reason2'],dst_col = 'reason',sep=','):
    sub_pairs[dst_col] = sub_pairs[cols[0]].str.cat(sub_pairs[cols[1]], sep=sep)
    return sub_pairs
#======================================================================================================================

def list2json(x):
    x = str(x)
    k = ''
    ltx = x.split(',')
    for item in ltx:
        if k != '':
            if item != '':
                k = k + ',' + "\"" + item + "\""
            else:
                pass
        else:
            if item != '':
                k = "\""+item+"\""
            else:
                pass
    k = '['+k+']'
    return k

def reason_json_format(df,col_name:str='reason'):
    df[col_name] = df[col_name].apply(lambda x: '{\"reasons\":' + list2json(x) + '}')
    return df

In [162]:
print('Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>')
print('Filtering')
#Global filter: master degree!
global_ft = global_filter(loc_feat=loc_feat)
sub_loc_feat = global_ft.city_filter(city_name=cityname[ind_city]).end()


if wework_location_only:
    sub_loc_feat_ww = sub_loc_feat[sub_loc_feat['is_wework']==True]
    sub_comp_loc = pd.merge(comp_loc,sub_loc_feat_ww[['atlas_location_uuid']],on='atlas_location_uuid',how='inner',suffixes=['','_right'])
else:
    sub_comp_loc = pd.merge(comp_loc, sub_loc_feat[['atlas_location_uuid']], on='atlas_location_uuid',
                            how='inner', suffixes=['', '_right'])
    
print('Reasoning')
#Reason 1:
print('Customized Reason')
matching_col = 'major_industry_category'
# matching_col = 'primary_sic_2_digit_v2'
recall_com1 = sub_rec_similar_company(comp_feat=comp_feat, comp_loc=sub_comp_loc,
                                      matching_col=matching_col,reason_col_name='reason')
sub_pairs = recall_com1.get_candidate_location_for_company(query_comp_feat=comp_feat,reason='like')
sub_pairs['reason'] = sub_pairs.apply(lambda x: 'suitable for '+ x[matching_col],axis=1)
print(sub_pairs.shape)

#Reason2:
print('General Reason')
recall_com2 = sub_rec_condition(sub_loc_feat)
sub_loc_recall_com2 = recall_com2.exfiltering('num_fitness_gyms', percentile=0.5, reason='Enough GYM')
sub_loc_recall_com3 = recall_com2.exfiltering('num_drinking_places', percentile=0.5,
                                              reason='Entertainment Available')
sub_loc_recall_com4 = recall_com2.exfiltering('num_eating_places', percentile=0.5, reason='Easy for lunch')
print('recall_location_size: %d, %d, %d'%(len(sub_loc_recall_com2), len(sub_loc_recall_com3), len(sub_loc_recall_com4.shape)))

print('Merging general reasons')
sub_loc_recall = pd.concat([sub_loc_recall_com2, sub_loc_recall_com3, sub_loc_recall_com4], axis=0)
sub_loc_recall = merge_rec_reason_rowise(sub_loc_recall, group_cols=['atlas_location_uuid'],
                                         merge_col='reason')
if wework_location_only:
    sub_loc_recall = sub_loc_recall.merge(sub_loc_feat_ww[['atlas_location_uuid']],on='atlas_location_uuid',how='inner',suffixes=['','_right'])
print('sub_loc_recall sized %d' % len(sub_loc_recall))
sub_loc_recall.head()
# print('Making pairs for general reasons')
# comp_feat['key'] = 0
# sub_loc_recall['key'] = 0
# sub_pairs2 = pd.merge(comp_feat[['duns_number', 'key']], sub_loc_recall, on='key', how='left',
#                            suffixes=['', '_right'])
# sub_pairs2.head()

Generate reason files:<key,reason>:<locid,reason>,<(compid,locid),reason>
Filtering
Reasoning
Customized Reason
(1268040, 4)
General Reason
recall_location_size: 580, 580, 2
Merging general reasons
sub_loc_recall sized 9


/Users/yefeichen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: FutureWarning: `item` has been deprecated and will be removed in a future version


,atlas_location_uuid,reason
0,1fdbb2a3-7583-43f5-8637-fc0a86776b27,"Enough GYM,Entertainment Available,Easy for lunch"
1,399e5683-c0c1-4f67-bdb3-1896df94164e,"Enough GYM,Entertainment Available,Easy for lunch"
2,3f540e10-8251-0130-f786-123138068411,"Enough GYM,Entertainment Available,Easy for lunch"
3,66760fc3-632c-4448-bce8-8660f987b0a5,"Enough GYM,Entertainment Available,Easy for lunch"
4,6f6f7830-9836-456f-a7ef-218c0db9581f,"Enough GYM,Entertainment Available,Easy for lunch"


1367179235150850978

In [163]:
#### attach these with scores
ssfile = ['ww_PA_similarity_20191113.csv','ww_SF_similarity_20191113.csv','ww_SJ_similarity_20191113.csv',
          'ww_LA_similarity_20191113.csv','ww_NY_similarity_20191113.csv']

topk=300
sspd = pd.read_csv(pjoin(datapath,ssfile[ind_city]),index_col=0)

sample_sspd = sspd.groupby('atlas_location_uuid').apply(lambda x: x.nlargest(topk,['similarity'])).reset_index(drop=True)

print(len(sample_sspd))

sample_sspd = pd.merge(sample_sspd,sub_loc_recall[['atlas_location_uuid','reason']],on='atlas_location_uuid',how='left',suffixes=['','_right']).reset_index(drop=True)

sample_sspd = pd.merge(sample_sspd,sub_pairs,on=['atlas_location_uuid','duns_number'],how='left',suffixes=['','_right'])

sample_sspd = sample_sspd[sample_sspd['reason'].notnull() | sample_sspd['reason_right'].notnull()]

sample_sspd[['reason','reason_right']] = sample_sspd[['reason','reason_right']].fillna('')

sample_sspd = merge_rec_reason_colwise(sample_sspd,cols=['reason','reason_right'],dst_col='reason')


sample_sspd = reason_json_format(sample_sspd,col_name='reason')

sample_sspd['duns_number'] = sample_sspd['duns_number'].astype(int)
sample_sspd = sample_sspd.rename(columns={
    "reason":"note","duns_number":"company_id"
})
sample_sspd['building_id'] = sample_sspd['atlas_location_uuid'].apply(lambda x: hash(x))
sample_sspd['algorithm']= 'model_deep_and_wide'

col_list = ['company_id','building_id','similarity','note','atlas_location_uuid','algorithm']
sample_sspd = sample_sspd[col_list]

print(len(sample_sspd))

sample_sspd.to_csv('sub_'+ssfile[ind_city])
# sspd[sspd['reason_right'].isnull()]

3900
3801


In [112]:
sample_sspd

,company_id,building_id,similarity,notes,atlas_location_uuid,algorithm
0,80363322,6155664384454325053,0.988245,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
1,94980976,6155664384454325053,0.987349,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
2,123591612,6155664384454325053,0.986365,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
3,21824848,6155664384454325053,0.985952,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
4,792250060,6155664384454325053,0.985291,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
5,13885336,6155664384454325053,0.984176,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
8,797051620,6155664384454325053,0.980513,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
9,76310226,6155664384454325053,0.980114,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
10,29276342,6155664384454325053,0.979620,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide
11,834706806,6155664384454325053,0.978484,"{""""""reason"""""":[""""""Enough GYM"""""",""""""Entertainme...",33ee4501-bd53-407f-81bd-7865a7b46423,model_deep_and_wide


In [85]:
sspd.head()
sspd[sspd['reason_right'].isnull()]

,atlas_location_uuid,duns_number,similarity,reason,major_industry_category,reason_right


In [51]:
sub_pairs['reason'] = sub_pairs.apply(lambda x: 'most '+ x['primary_sic_2_digit_v2'] + ' ' + x['reason'],axis=1)
sub_pairs.head()

,duns_number,primary_sic_2_digit_v2,atlas_location_uuid,reason
0,105571242,"ENGINEERING, ACCOUNTING, RESEARCH, AND MANAGEM...",6ba62017-ae70-43c3-afa0-cb4459fe51da,"most ENGINEERING, ACCOUNTING, RESEARCH, AND MA..."
1,555658061,BUSINESS SERVICES,6ba62017-ae70-43c3-afa0-cb4459fe51da,most BUSINESS SERVICES BUSINESS SERVICES BUSIN...
3,62393456,"ENGINEERING, ACCOUNTING, RESEARCH, AND MANAGEM...",6ba62017-ae70-43c3-afa0-cb4459fe51da,"most ENGINEERING, ACCOUNTING, RESEARCH, AND MA..."
5,159329432,EDUCATIONAL SERVICES,6ba62017-ae70-43c3-afa0-cb4459fe51da,most EDUCATIONAL SERVICES EDUCATIONAL SERVICES...
6,79081783,HEALTH SERVICES,6ba62017-ae70-43c3-afa0-cb4459fe51da,most HEALTH SERVICES HEALTH SERVICES HEALTH SE...


In [138]:
#### attach these with scores
ssfile = ['ww_PA_similarity_20191113.csv','ww_SF_similarity_20191113.csv','ww_SJ_similarity_20191113.csv',
          'ww_LA_similarity_20191113.csv','ww_NY_similarity_20191113.csv']

topk=300
sspd = pd.read_csv(pjoin(datapath,ssfile[ind_city]),index_col=0)

sample_sspd = sspd.groupby('atlas_location_uuid').apply(lambda x: x.nlargest(topk,['similarity'])).reset_index(drop=True)

print(len(sample_sspd))

sample_sspd = pd.merge(sample_sspd,sub_loc_recall[['atlas_location_uuid','reason']],on='atlas_location_uuid',how='left',suffixes=['','_right']).reset_index(drop=True)

sample_sspd = pd.merge(sample_sspd,sub_pairs,on=['atlas_location_uuid','duns_number'],how='left',suffixes=['','_right'])

23100


In [144]:
sample_sspd[['reason','reason_right']] = sample_sspd[['reason','reason_right']].fillna('')

In [145]:
sample_sspd2 = merge_rec_reason_colwise(sample_sspd,cols=['reason','reason_right'],dst_col='reason')

In [146]:
sample_sspd2

,atlas_location_uuid,duns_number,similarity,reason,major_industry_category,reason_right
0,03acc90a-4434-460d-bcba-cfee013c44d0,107212169.0,0.978269,",",NaN,
1,03acc90a-4434-460d-bcba-cfee013c44d0,197074024.0,0.925308,",suitable for MANUFACTURING",MANUFACTURING,suitable for MANUFACTURING
2,03acc90a-4434-460d-bcba-cfee013c44d0,556285401.0,0.924801,",",NaN,
3,03acc90a-4434-460d-bcba-cfee013c44d0,845833446.0,0.924738,",",NaN,
4,03acc90a-4434-460d-bcba-cfee013c44d0,80327247.0,0.924298,",",NaN,
5,03acc90a-4434-460d-bcba-cfee013c44d0,844044144.0,0.922188,",",NaN,
6,03acc90a-4434-460d-bcba-cfee013c44d0,828844535.0,0.920518,",",NaN,
7,03acc90a-4434-460d-bcba-cfee013c44d0,169756272.0,0.919781,",",NaN,
8,03acc90a-4434-460d-bcba-cfee013c44d0,27549649.0,0.919449,",",NaN,
9,03acc90a-4434-460d-bcba-cfee013c44d0,150131683.0,0.918485,",",NaN,


In [211]:
##merging files
dfs = []
for filename in ssfile:
    dfs.append(pd.read_csv('sub_'+filename,index_col=0))
dfs = pd.concat(dfs,axis=0).reset_index(drop=True)

loc_df = dfs.groupby('atlas_location_uuid')[['atlas_location_uuid']].first().reset_index(drop=True)

# if 'notes' in dfs.columns:
#     dfs.rename(columns={
#     "notes":"note"})




In [212]:
k = list(range(len(loc_df)))
pd_id = pd.DataFrame(np.array(k),columns=['building_id'])
loc_df = pd.concat([loc_df,pd_id],axis=1)

dfs = dfs[['company_id', 'similarity', 'note', 'algorithm', 'atlas_location_uuid']].merge(loc_df,on='atlas_location_uuid',how='left',suffixes=['','_right'])

col_list = ['company_id', 'building_id', 'similarity', 'note', 'algorithm', 'atlas_location_uuid']
dfs = dfs[col_list]

dfs.to_csv('sub_all_similarity.csv',index=False)

In [213]:
dfs

,company_id,building_id,similarity,note,algorithm,atlas_location_uuid
0,191422398,49,0.987909,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
1,14896204,49,0.966687,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
2,961376738,49,0.966483,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
3,98533516,49,0.963027,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
4,21082220,49,0.962931,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
5,178928479,49,0.962574,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
6,68633735,49,0.962422,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
7,944550750,49,0.962422,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
8,10545975,49,0.962422,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
9,168264414,49,0.960006,"{""reasons"":[""Enough GYM"",""Entertainment Availa...",model_deep_and_wide,6ba62017-ae70-43c3-afa0-cb4459fe51da
